In [541]:
import collections
import datetime
import itertools as it
import math
import numpy as np
import pandas as pd

import cric
reload(cric.inning)
reload(cric.match)
reload(cric.pic)

<module 'cric.pickers' from 'cric/pickers.pyc'>

In [534]:
summaries = cric.match_summaries_from_dir('data/t20s')
matches_df = cric.match.create_match_summaries_frame(it.islice(summaries, 200))

In [535]:
innings = cric.match.pic.pick_innings(it.islice(summaries, 1).next())
inn1_data = innings[0].items()[0][1]
inn2_data = innings[1].items()[0][1]

In [536]:
ifr = cric.inning.create_innings_dataframe(1, inn1_data)
ifr.fillna('-').head(18)

batsman     bowler  non_striker  rb  re  rt  rb_cum  \
inning over ball                                                            
1      0.0  0.1   BB McCullum    SW Tait    PJ Ingram   0   4   4       0   
            0.2   BB McCullum    SW Tait    PJ Ingram   0   0   0       0   
            0.3   BB McCullum    SW Tait    PJ Ingram   4   0   4       4   
            0.4   BB McCullum    SW Tait    PJ Ingram   1   0   1       5   
            0.5     PJ Ingram    SW Tait  BB McCullum   0   1   1       0   
            0.6     PJ Ingram    SW Tait  BB McCullum   0   0   0       0   
            0.7     PJ Ingram    SW Tait  BB McCullum   0   0   0       0   
       1.0  0.1   BB McCullum  DP Nannes   MJ Guptill   6   0   6      11   
            0.2   BB McCullum  DP Nannes   MJ Guptill   3   0   3      14   
            0.3    MJ Guptill  DP Nannes  BB McCullum   1   0   1       1   
            0.4   BB McCullum  DP Nannes   MJ Guptill   0   0   0      14   
            0.5   BB McCullum  DP Nannes   MJ Guptill   4   0   4      18   
            0.6   BB McCullum  DP Nannes   MJ Guptill   0   0   0      18   
       2.0  0.1    MJ Guptill    SW Tait  BB McCullum   0   0   0       1   
            0.2    MJ Guptill    SW Tait  BB McCullum   0   0   0       1   
            0.3    MJ Guptill    SW Tait  BB McCullum   0   0   0       1   
            0.4    MJ Guptill    SW Tait  BB McCullum   0   0   0       1   
            0.5    MJ Guptill    SW Tait  BB McCullum   4   0   4       5   

                  re_cum  rt_cum wf      wk        wpo  w_cum  
inning over ball                                               
1      0.0  0.1        4       4  -       -          -      0  
            0.2        4       4  -       -          -      0  
            0.3        4       8  -       -          -      0  
            0.4        4       9  -       -          -      0  
            0.5        5      10  -       -          -      0  
            0.6        5      10  -       -          -      0  
            0.7        5      10  -  bowled  PJ Ingram      1  
       1.0  0.1        5      16  -       -          -      1  
            0.2        5      19  -       -          -      1  
            0.3        5      20  -       -          -      1  
            0.4        5      20  -       -          -      1  
            0.5        5      24  -       -          -      1  
            0.6        5      24  -       -          -      1  
       2.0  0.1        5      24  -       -          -      1  
            0.2        5      24  -       -          -      1  
            0.3        5      24  -       -          -      1  
            0.4        5      24  -       -          -      1  
            0.5        5      28  -       -          -      1

In [537]:
matches_df.iloc[0]['Australia'].head(10)

batsman           bowler non_striker  rb  re  rt  rb_cum  \
inning over ball                                                               
1      0.0  0.1    AJ Finch       SL Malinga   M Klinger   0   0   0       0   
            0.2    AJ Finch       SL Malinga   M Klinger   0   0   0       0   
            0.3    AJ Finch       SL Malinga   M Klinger   1   0   1       1   
            0.4   M Klinger       SL Malinga    AJ Finch   2   0   2       2   
            0.5   M Klinger       SL Malinga    AJ Finch   0   0   0       2   
            0.6   M Klinger       SL Malinga    AJ Finch   3   0   3       5   
       1.0  0.1   M Klinger  KMDN Kulasekara    AJ Finch   0   0   0       5   
            0.2   M Klinger  KMDN Kulasekara    AJ Finch   1   0   1       6   
            0.3    AJ Finch  KMDN Kulasekara   M Klinger   0   0   0       1   
            0.4    AJ Finch  KMDN Kulasekara   M Klinger   0   0   0       1   

                  re_cum  rt_cum   wf   wk  wpo  w_cum  
inning over ball                                        
1      0.0  0.1        0       0  NaN  NaN  NaN      0  
            0.2        0       0  NaN  NaN  NaN      0  
            0.3        0       1  NaN  NaN  NaN      0  
            0.4        0       3  NaN  NaN  NaN      0  
            0.5        0       3  NaN  NaN  NaN      0  
            0.6        0       6  NaN  NaN  NaN      0  
       1.0  0.1        0       6  NaN  NaN  NaN      0  
            0.2        0       7  NaN  NaN  NaN      0  
            0.3        0       7  NaN  NaN  NaN      0  
            0.4        0       7  NaN  NaN  NaN      0

In [538]:
matches_df = matches_df.sort_index(level='dates')
matches_df = matches_df[matches_df.gender == 'male']

#print matches_df.columns[:1]
#matches_df.xs('India', level='team.home', drop_level=False).head().T

In [544]:
idx, match = it.islice(matches_df.iterrows(), 1).next()
date, home_team, away_team = idx
first_match = cric.match.flat_match(date, home_team, away_team, match)
all_matches = pd.concat((cric.match.flat_match(*idx, match_s=match) for (idx, match) in matches_df.iterrows()), axis=0)
all_matches.columns

Index([u'1st innings', u'batsman', u'bowler', u'city', u'match_type',
       u'neutral_venue', u'non_striker', u'overs', u'rb', u'rb_cum', u're',
       u're_cum', u'replacements', u'rt', u'rt_cum', u'supersubs',
       u'toss.decision', u'toss.winner', u'umpires', u'venue', u'w_cum', u'wf',
       u'wk', u'wpo'],
      dtype='object')

In [551]:
all_matches.fillna('-').tail(20).T

date                        2017-06-05                           \
team.home                  West Indies                            
team.away                  Afghanistan                            
inning                               2                            
over                              16.0                            
ball                               0.1                      0.2   
1st innings          Afghanistan 146/6        Afghanistan 146/6   
batsman                    JN Mohammed               MN Samuels   
bowler                     Rashid Khan              Rashid Khan   
city                          St Kitts                 St Kitts   
match_type                         T20                      T20   
neutral_venue                        -                        -   
non_striker                 MN Samuels              JN Mohammed   
overs                               20                       20   
rb                                   1                        1   
rb_cum                              10                       77   
re                                   0                        0   
re_cum                               1                        1   
replacements                         -                        -   
rt                                   1                        1   
rt_cum                             121                      122   
supersubs                            -                        -   
toss.decision                        -                        -   
toss.winner                Afghanistan              Afghanistan   
umpires         [LS Reifer, JS Wilson]   [LS Reifer, JS Wilson]   
venue          Warner Park, Basseterre  Warner Park, Basseterre   
w_cum                                3                        3   
wf                                   -                        -   
wk                                   -                        -   
wpo                                  -                        -   

date                                                             \
team.home                                                         
team.away                                                         
inning                                                            
over                                                              
ball                               0.3                      0.4   
1st innings          Afghanistan 146/6        Afghanistan 146/6   
batsman                    JN Mohammed               MN Samuels   
bowler                     Rashid Khan              Rashid Khan   
city                          St Kitts                 St Kitts   
match_type                         T20                      T20   
neutral_venue                        -                        -   
non_striker                 MN Samuels              JN Mohammed   
overs                               20                       20   
rb                                   1                        0   
rb_cum                              11                       77   
re                                   0                        0   
re_cum                               1                        1   
replacements                         -                        -   
rt                                   1                        0   
rt_cum                             123                      123   
supersubs                            -                        -   
toss.decision                        -                        -   
toss.winner                Afghanistan              Afghanistan   
umpires         [LS Reifer, JS Wilson]   [LS Reifer, JS Wilson]   
venue          Warner Park, Basseterre  Warner Park, Basseterre   
w_cum                                3                        3   
wf                                   -                        -   
wk                                   -                        -   
wpo                                  -        

In [556]:
all_matches.groupby(['batsman'])['rb'].rolling(12, min_periods=2).sum().sort_index(level=['date', 'inning', 'over', 'ball'])

batsman       date        team.home    team.away    inning  over  ball
AC Gilchrist  2005-02-17  New Zealand  Australia    1       0.0   0.1      NaN
                                                                  0.2      0.0
MJ Clarke     2005-02-17  New Zealand  Australia    1       0.0   0.3      NaN
                                                                  0.4      1.0
AC Gilchrist  2005-02-17  New Zealand  Australia    1       0.0   0.5      1.0
MJ Clarke     2005-02-17  New Zealand  Australia    1       0.0   0.6      7.0
                                                                  0.7      7.0
A Symonds     2005-02-17  New Zealand  Australia    1       1.0   0.1      NaN
                                                                  0.2      4.0
                                                                  0.3     10.0
                                                                  0.4     11.0
AC Gilchrist  2005-02-17  New Zealand  Australia    1       

In [351]:
df1 = pd.DataFrame({'key':[1,2], 'col1':[1,2],'col2':[3,4]})
df2 = pd.DataFrame({'key':[1,1,2,2], 'col3':[5,6,7,8]})

pd.merge(df1, df2, on='key', how='left')

,col1,col2,key,col3
0,1,3,1,5
1,1,3,1,6
2,2,4,2,7
3,2,4,2,8
